In [1]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json
from sklearn.preprocessing import StandardScaler

dataframe = pd.read_csv("DatosScaler.csv")
dataframe = dataframe.drop(['Unnamed: 0'], axis=1)
datos = dataframe.values
scaler = StandardScaler()
scaler.fit(datos)


args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'custom' # data
args.root_path = './data/ETT/' # root path of data file
args.data_path = 'Datos.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'X54' # target feature in S or MS task
args.freq = 'h' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './checkpoints' # location of model checkpoints

args.seq_len = 96 # input sequence length of Informer encoder
args.label_len = 48 # start token length of Informer decoder
args.pred_len = 24 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 3 # encoder input size
args.dec_in = 3 # decoder input size
args.c_out = 3 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 'h'

args.batch_size = 32
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'
Exp = Exp_Informer  #se llama al modelo
setting = 'informer_custom_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_test_0' #dirección dónde se encuentra guardado el mejor peso

exp = Exp(args) # se crea experimenmto junto a los argumentos (argumentos entran al modelo)
exp.predict(setting, True)
pronóstico = np.load('./results/'+setting+'/real_prediction.npy') #se carga pronóstico desde archivo .npy
scaler_class = scaler.transform(pronóstico)
v = scaler_class.reshape((scaler_class.shape[0], scaler_class.shape[1], 1)) #se prepara la entrada para el siguiente modelo
optimizer=keras.optimizers.Adam(learning_rate=1e-3)
json_file = open('model_guardado5.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# cargar los pesos del nuevo modelo
loaded_model.load_weights("weights.best5.hdf5")
print("Cargar modelo del disco")
# evaluar el modelo cargado desde el disco con los datos 
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)
clasificador = loaded_model.predict(v)
folder_path = './results/' + setting +'/'
np.save(folder_path+'real_class.npy', clasificador)


Use GPU: cuda:0
pred 1
Cargar modelo del disco


In [5]:
def informacion():
    
    global anomalias, anomalias_corrientes, anomalias_vibla, anomalias_vibll
    pronóstico = np.load('./results/'+setting+'/real_prediction.npy') #se carga pronóstico desde archivo .npy
    categoria=np.load('./results/'+setting+'/real_class.npy') #se carga pronóstico desde archivo .npy
    pred_category = np.zeros(24)
    i=0
    while i<=23:
        pred_category[i] = np.argmax(categoria[i])
        i = i+1 

    x1 = list(pronóstico[:, 0])
    x2 = list(pronóstico[:, 1])
    x3 = list(pronóstico[:, 2])
    x4 = list(pred_category)
    tabla=[x1]+[x2]+[x3]+[x4]
    tabla=np.transpose(tabla)
    columnas = ['Corriente','vibla','vibll','anomalias']
    tabla = pd.DataFrame(tabla, columns=columnas)
    anomalias= tabla.loc[tabla.anomalias == True]
    anomalias_corrientes = tabla.loc[(tabla.Corriente > 274) | (tabla.Corriente < 253)]
    anomalias_vibla = tabla.loc[(tabla.vibla > 2.85) | (tabla.vibla <  1.99)]
    anomalias_vibll = tabla.loc[(tabla.vibll > 2.18) | (tabla.vibll < 1.2)]


In [ ]:
n_anomalias = len(anomalias_corrientes) + len(anomalias_vibla) + len(anomalias_vibll)